<a href="https://colab.research.google.com/github/AdrianaUP/News-query_RPP-lab/blob/main/notebooks/task1_rpp_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Notebook inicializado para Task 1")


Notebook inicializado para Task 1
